# thermoelectric_experiment

In [55]:
import json

with open('ucsb_thermoelectrics.json', 'r', encoding='utf-8') as f:
    thermo = json.load(f)
dft_zt_dict = {}
for d in thermo['data']:
    com = d[0]
    # if zt, use d[7], if pf, use d[6]
    zt = d[6]
    if pd.isna(zt):
        continue
    if com not in dft_zt_dict.keys():
        dft_zt_dict[com] = [zt]
    else:
        dft_zt_dict[com].append(zt)
sorted_dft = sorted(dft_zt_dict.items(), key=lambda item:max(item[1]), reverse=True)

with open('experiment_pf.txt', 'w', encoding='utf-8') as f:
    for s in sorted_dft:
        f.write(s[0]+'\t'+str(max(dft_zt_dict[s[0]]))+'\n')

# thermoelectric_dft

In [3]:
import json

with open('thermoelectric_dft/TE_data_used_in_paper.json', 'r', encoding='utf-8') as f:
    thermo = json.load(f)
    
normalized_dft_dict = {}
dft_dict = {}
for mat in thermo.keys():
    if thermo[mat][2]:
        normalized_dft_dict[mat] = thermo[mat][2][0]
        all_names = list(thermo[mat][0].keys())
        dft_dict[all_names[0]] = thermo[mat][2][0]
        
sorted_dft_n = sorted(normalized_dft_dict.items(), key=lambda item:item[1], reverse=True)
with open('thermoelectric_dft/dft_normalized.txt', 'w', encoding='utf-8') as f:
    for s in sorted_dft_n:
        f.write(s[0]+'\t'+str(s[1])+'\n')

sorted_dft = sorted(dft_dict.items(), key=lambda item:item[1], reverse=True)
with open('thermoelectric_dft/dft.txt', 'w', encoding='utf-8') as f:
    for s in sorted_dft:
        f.write(s[0]+'\t'+str(s[1])+'\n')

# perovskite

In [1]:
import pandas as pd

df = pd.read_csv('D:/个人/material+NLP/Passivating_contact/Perovskite_database_content_all_data.csv')
# 1. only get 4 key columns
etl = list(df['ETL_stack_sequence'])
htl = list(df['HTL_stack_sequence'])
per_short = list(df['Perovskite_composition_short_form'])
per_long = list(df['Perovskite_composition_long_form'])
pce = list(df['JV_default_PCE'])


D:\Anaconda3\envs\mat_emb\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10,22,29,31,32,35,36,40,44,45,46,48,51,54,65,84,89,90,93,98,99,100,105,108,115,118,122,123,125,130,134,138,142,143,144,146,149,152,163,166,167,171,172,173,175,178,181,192,194,225,271,272,273,277,304,315,321,325,330,331,335,336,342,348,369,371,373,374,376,380,384,387,403,405,407,409) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def normalize(split_m):
    if '-' in split_m:
        if split_m.count('-') == 1:
            idx_ = split_m.index('-')
            return split_m[:idx_]
    return split_m

In [45]:
etl_dict = {}
# htl or etl
for i, e in enumerate(htl):    
    if pd.isna(pce[i]):
        continue
    split_e = e.split(' | ')
    for se in split_e:
        if '; ' not in se:
            norm = normalize(se)
            if norm not in etl_dict.keys():
                etl_dict[norm] = {}
                etl_dict[norm]['score'] = [pce[i]]
                etl_dict[norm]['origin'] = [se]
            else:
                etl_dict[norm]['score'].append(pce[i])
                etl_dict[norm]['origin'].append(se)
        else:
            for ss in se.split('; '):
                norm = normalize(ss)
                if norm not in etl_dict.keys():
                    etl_dict[norm] = {}
                    etl_dict[norm]['score'] = [pce[i]]
                    etl_dict[norm]['origin'] = [ss]
                else:
                    etl_dict[norm]['score'].append(pce[i])
                    etl_dict[norm]['origin'].append(ss)
sorted_etl = sorted(etl_dict.items(), key=lambda item:max(item[1]['score']), reverse=True)

In [46]:
with open('htl_simplest_score.txt', 'w', encoding='utf-8') as f:
    for s in sorted_etl:
        if s[0]!='none' and s[0]!='Unknown':
            f.write(s[0]+'\t'+str(max(etl_dict[s[0]]['score']))+'\n')

In [13]:
# per_short or per_long
per_dict = {}
for i, e in enumerate(per_long): 
    if pd.isna(pce[i]):
        continue
    if '|' in e:
        split_e = e.split(' | ')
        for se in split_e:
            se = se.replace('nan', '')
            # print(se)
            if se != '':
                if se not in per_dict.keys():
                    per_dict[se] = [pce[i]]
                else:
                    per_dict[se].append(pce[i])
    else:
        e = e.replace('nan', '')
        # print(e)
        if e != '':
            if e not in per_dict.keys():
                per_dict[e] = [pce[i]]
            else:
                per_dict[e].append(pce[i])
    
sorted_per = sorted(per_dict.items(), key=lambda item:max(item[1]), reverse=True)
# for s in sorted_per:
    # print(s[0], max(per_dict[s[0]]))
    
with open('perovskite/per_long_simplest_score.txt', 'w', encoding='utf-8') as f:
    for s in sorted_per:
        if s[0]!='none' and s[0]!='Unknown' and max(per_dict[s[0]])!='nan':
            f.write(s[0]+'\t'+str(max(per_dict[s[0]]))+'\n')

In [12]:
# 2. remove repeated value and only reserve the combination with highest pce value
sen = []
for i, e in enumerate(etl):    
    # s = ''
    # s = 
    print(e, htl[i], per_short[i], per_long[i], pce[i])

TiO2-c | TiO2-mp Spiro-MeOTAD CsSnI CsSnI3 0.0
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBrI CsSnBr0.3I2.7 0.0
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBrI CsSnBr1.5I1.5 0.13
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBrI CsSnBr2.7I0.3 0.12
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBr CsSnBr3 0.1
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnI CsSnI3 1.66
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBrI CsSnBrI2 1.67
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBrI CsSnBr2I 1.56
TiO2-c | TiO2-mp Spiro-MeOTAD CsSnBr CsSnBr3 0.95
TiO2-c | TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 10.3
TiO2-c | TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 11.5
TiO2-c | TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 12.9
TiO2-c | TiO2-np none MAPbBr MAPbBr3 5.2
TiO2-c | TiO2-np none MAPbBr MAPbBr3 6.2
TiO2-c | TiO2-np Spiro-MeOTAD MAPbBr MAPbBr3 6.2
PCBM-70 | BCP PEDOT:PSS MAPbI MAPbI3 12.0
PCBM-70 | BCP PEDOT:PSS MAPbI MAPbI3 9.0
PCBM-70 | BCP PEDOT:PSS MAPbI MAPbI3 12.03
PCBM-70 | BCP PEDOT:PSS MAPbI MAPbI3 8.61
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 10.0
TiO2-c none MAPbI MAPbI3 0.6
TiO2-c Spiro-MeOTAD MA

TiO2-c | SrTiO3-np; TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 10.18
TiO2-c | SrTiO3-np; TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 8.29
TiO2-c | SrTiO3-np; TiO2-nw Spiro-MeOTAD MAPbI MAPbI3 7.41
PCBM-60 PEDOT:PSS MAPbI MAPbI3 13.51
PCBM-60 Nafion; PEDOT:PSS MAPbI MAPbI3 15.03
PCBM-60 Nafion; PEDOT:PSS MAPbI MAPbI3 16.72
PCBM-60 Nafion; PEDOT:PSS MAPbI MAPbI3 11.23
TiO2-c | TiO2-mp none MAPbI MAPbI3 1.55
TiO2-c | TiO2-mp none MAPbI MAPbI3 3.73
TiO2-c | TiO2-mp none MAPbI MAPbI3 7.83
TiO2-c | TiO2-mp | ZrO2-mp none MAPbI MAPbI3 12.68
TiO2-c | TiO2-mp | ZrO2-mp none MAPbI MAPbI3 13.89
PCBM-60 | ZnO PEDOT:PSS MAPbI MAPbI3 14.6
TiO2-c | TiO2-mp Spiro-MeOTAD FAMAPbBrI FA0.85MA0.15PbBr0.45I2.55 17.86
TiO2-c | TiO2-mp Spiro-MeOTAD FAMAPbBrI FA0.85MA0.15PbBr0.45I2.55 17.28
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 12.8
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 9.8
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 14.1
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 16.1
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 15.0
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 

PCBM-60 | BCP PEDOT:PSS (PEA)MAPbI (PEA)2MA3Pb4I13 14.1
C60 | BCP PTAA FAMAPbBrI FA0.97MA0.03PbBr0.09I2.91 19.66
C60 | BCP PTAA FAMAPbBrI FA0.97MA0.03PbBr0.09I2.91 20.93
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 18.22
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 18.22
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.025MA0.975PbI3 18.95
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.05MA0.95PbI3 19.6
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.075MA0.925PbI3 19.86
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.075MA0.925PbI3 19.86
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.1MA0.9PbI3 17.71
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.2MA0.8PbI3 15.21
TiO2-c | TiO2-mp Spiro-MeOTAD (THM)MAPbI (THM)0.2MA0.8PbI3 15.21
TiO2-c | TiO2-mp PTAA FAMAPbBrI FA0.88MA0.12PbBr0.36I2.64 21.06
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 13.67
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 13.49
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 13.45
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 6.63
TiO2-

TiO2-c Spiro-MeOTAD MAPbI MAPbI3 18.36
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 15.29
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 19.4
TiO2-c PTB7 MAPbI MAPbI3 7.53
TiO2-c PTB7 MAPbI MAPbI3 10.17
TiO2-c P3HT MAPbI MAPbI3 8.49
TiO2-c P3HT MAPbI MAPbI3 11.05
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 9.5
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 10.0
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 15.9
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 10.0
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 7.2
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 10.4
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 10.1
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 5.0
TiO2-c Spiro-MeOTAD CsPbI CsPbI3 12.15
TiO2-c Spiro-MeOTAD CsFAPbI | CsPbI Cs0.25FA0.75PbI3 | CsPbI3 15.52
TiO2-c Spiro-MeOTAD CsFAPbI | CsPbI Cs0.25FA0.75PbI3 | CsPbI3 13.11
TiO2-c Spiro-MeOTAD CsFAPbI | CsPbI Cs0.25FA0.75PbI3 | CsPbI3 7.45
TiO2-c Spiro-MeOTAD CsFAPbI Cs0.25FA0.75PbI3 6.94
TiO2-c Spiro-MeOTAD CsPbI | CsPbI CsPbI3 | CsPbI3 12.15
TiO2-c Spiro-MeOTAD CsFAPbI | CsPbI Cs0.75FA0.25PbI3 | CsPbI3 15.74
TiO2-c Spiro-MeOTAD CsFAPbI | CsPbI Cs0.5FA0.5PbI3 | 

PCBM-70 | BCP NiO-c MAPbI MAPbI3 15.04
PCBM-70 | BCP PEDOT:PSS MAPbI MAPbI3 10.24
PCBM-70 | BCP NiO-c MAPbI MAPbI3 11.4
PCBM-70 | BCP NiO-c MAPbI MAPbI3 13.14
PCBM-70 | BCP NiO-c MAPbI MAPbI3 14.06
PCBM-70 | BCP NiO-c MAPbI MAPbI3 13.06
PCBM-70 | BCP NiO-c MAPbI MAPbI3 14.06
PCBM-70 | BCP NiO-c MAPbI MAPbI3 16.86
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 16.85
C60 | BCP PEDOT:PSS MAPbI MAPbI3 14.31
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 14.39
C60 | BCP PEDOT:PSS MAPbI MAPbI3 13.69
C60 | BCP VOx MAPbI MAPbI3 11.86
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 13.63
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 14.47
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 15.98
C60 | BCP VOx | Cu phtalocyanine MAPbI MAPbI3 13.45
PCBM-60 | TOPD MoOx | PEDOT:PSS MAPbI MAPbI3 16.04
PCBM-60 | TOPD MoOx | PEDOT:PSS MAPbI MAPbI3 16.04
PCBM-60 | TOPD MoOx | PEDOT:PSS MAPbI MAPbI3 16.04
PCBM-60 | TOPD MoOx | PEDOT:PSS MAPbI MAPbI3 16.04
PCBM-60 | TOPD MoOx | PEDOT:PSS MAPbI MAPbI3 16.04
PCBM-60

...

TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 15.64
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 12.0
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 12.5
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 14.91
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 10.0
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 7.5
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.8MA0.2PbBr0.22I 18.1
C60 | BCP Poly(3-bromothiophene) MAPbBrI MAPbBr0.4I2.6 16.0
C60 | BCP Poly(3-bromothiophene) MAPbBrI MAPbBr0.4I2.6 16.5
C60 | BCP Poly(3-bromothiophene) MAPbBrI MAPbBr0.4I2.6 17.3
C60 | BCP Poly(3-bromothiophene) MAPbBrI MAPbBr0.4I2.6 15.6
C60 | BCP Poly(3-bromothiophene) MAPbBrI MAPbBr0.4I2.6 14.5
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.4MA0.6PbBr0.6I2.4 13.1
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.4MA0.6PbBr0.6I2.4 13.6
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.6MA0.4PbBr0.4I2.6 16.0
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.6MA0.4PbBr0.4I2.6 16.7
C60 | BCP Poly(3-bromothiophene) FAMAPbBrI FA0.8MA0.2PbBr0.2I2.

TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPb1.0I3 17.8
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPb1.0I3 17.6
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPb1.0I3 15.9
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPb1.0I3 13.1
PCBM-60 PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 10.64
PCBM-60 | BCP PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 9.6
PCBM-60 | BCP PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 12.39
PCBM-60 | BCP PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 10.81
PCBM-60 | BCP PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 13.06
PCBM-60 | BCP PEDOT:PSS MAPbBrI MAPb1.0Br0.6I2.4 10.73
PCBM-60 PEDOT:PSS MAPbI MAPb1.0I3 4.55
PCBM-60 PEDOT:PSS MAPbI MAPb1.0I3 7.72
PCBM-60 PEDOT:PSS MAPbI MAPb1.0I3 8.74
PCBM-60 PEDOT:PSS MAPbI MAPb1.0I3 10.23
PCBM-60 NiO-np MAPbI MAPb1.0I3 15.6
PCBM-60 | ZnO-np NiO-np MAPbI MAPb1.0I3 16.4
PCBM-60 NiO-np MAPbI MAPb1.0I3 14.5
PCBM-60 | ZnO-np NiO-np MAPbI MAPb1.0I3 14.9
PCBM-60 | ZnO-np NiO-np MAPbI MAPb1.0I3 16.4
PCBM-60 | ZnO-np NiO-np CsFAMAPbBrI Cs0.05FA0.79MA0.160Pb1.0Br0.3I2.7 17.7
PCBM-60 | ZnO-np NiO-np MAPbI MAPb1

ZnO-c Spiro-MeOTAD MAPbI MAPbI3 8.2
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 4.1
ZnO-c Spiro-MeOTAD MAPbI MAPbI3 14.2
PCBM-60 PEDOT:PSS MAPbI MAPbI3 11.6
TiO2-np NiO-np | Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.45I2.55 18.51
TiO2-np NiO-np | Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.45I2.55 18.11
TiO2-np NiO-np | Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.45I2.55 19.89
TiO2-np NiO-np | Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.45I2.55 17.8
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 8.9
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 10.3
TiO2-c Spiro-MeOTAD MAPbI MAPbI3 11.12
TiO2-c | ZnCdS Spiro-MeOTAD MAPbI MAPbI3 12.02
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 9.23
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 7.16
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 10.75
TiO2-nanofibers Spiro-MeOTAD MAPbI MAPbI3 11.16
TiO2-c | TiO2-nanofibres Spiro-MeOTAD MAPbI MAPbI3 14.92
TiO2-c | TiO2-nanofibres Spiro-MeOTAD MAPbI MAPbI3 10.27
TiO2-c | TiO2-nanofibres Spiro-MeOTAD MAPbI MAPbI3 10.04
T

TiO2-c Spiro-MeOTAD CsFAMAPbBrI Cs0.02FA0.83MA0.17PbBr0.51I2.49 12.6
TiO2-c Spiro-MeOTAD CsFAMAPbBrI Cs0.02FA0.83MA0.17PbBr0.51I2.49 13.8
TiO2-c Spiro-MeOTAD CsFAMAPbBrI Cs0.02FA0.83MA0.17PbBr0.51I2.49 13.5
TiO2-c Spiro-MeOTAD CsFAMAPbBrI Cs0.02FA0.83MA0.17PbBr0.51I2.49 12.7
TiO2-c Spiro-MeOTAD CsFAMAPbBrI Cs0.02FA0.83MA0.17PbBr0.51I2.49 12.7
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 14.22
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 15.58
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 16.53
TiO2-c | TiO2-mp Spiro-MeOTAD MAPbI MAPbI3 16.26
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.06FA0.79MA0.15PbBr0.45I2.55 17.2
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.06FA0.79MA0.15PbBr0.45I2.55 21.3
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.06FA0.79MA0.15PbBr1.8I1.2 11.7
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.06FA0.79MA0.15PbBr1.8I1.2 17.1
PCBM-60 | BCP PTAA MAPbI MAPbI3 15.79
PCBM-60 | BCP PTAA MAPbI MAPbI3 14.08
C60; PCBM-60 | bis-C60 NiO MAPbI MAPbI3 15.43
C60; PCBM-60 | bis-C60 Ni

TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.5I2.5 12.83
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.5I2.5 17.63
TiO2-c | TiO2-mp Spiro-MeOTAD CsFAMAPbBrI Cs0.05FA0.79MA0.16PbBr0.5I2.5 17.45
TiO2-c PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 16.5
TiO2-c PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 16.5
TiO2-c | PCBM-60 PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 17.1
TiO2-c | PCBM-60 PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 17.1
TiO2-c | C60-BCT-Au-NP PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 18.7
TiO2-c | C60-BCT-Au-NP PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 18.7
TiO2-c | C60-BCT PTAA CsFAMAPbBrI Cs0.05FA0.7885MA0.1615PbBr0.51I2.49 16.9
PCBM-60 | ZnO NiO-np CsFAMAPbBrI Cs0.175FA0.750MA0.075PbBr0.36I2.64 19.9
PCBM-60 | ZnO NiO-np CsFAMAPbBrI Cs0.175FA0.750MA0.075PbBr0.36I2.64 18.2
SnO2-np Spiro-MeOTAD FAMAPbBrI FAxMAxPbBrxIx 17.7
TiO2-c | TiO2-mp Spiro-MeOTAD FAMAPbBrI FA0.97MA0.03P

In [5]:
collect = []
for i, e in enumerate(etl):
    for element in e.split():
        if '-' in element:
            if element.split('-')[1] not in collect:
                collect.append(element.split('-')[1])
            
    # print(e, htl[i], per[i], pce[i])

In [6]:
print(collect)

['c', 'mp', 'nw', 'np', '70', '60', 'QDs', 'nt', 'HATNA', 'C60', 'CBL', 'np;', 'lactone', 'P', 'SAM', '60;', 'PCBM', '3CN', 'QDs;', 'IO', 'H', 'Cl', 'Br', 'V', '2CN', '1', 'NaDPO:Sn(SCN)2', 'NaDPO', '3N', 'ns', 'nt;', 'diaminoethane', 'Py', 'PDI', 'hollow', 'M', '2', '3', '4', 'Alanine', 'dimethyloxamic', 'PDTON', 'nanofibers', 'Th', 'Cou', '(1', 'POM', 'nanoflowers', '43', 'Carbon', '70;', 'T', 'Tz', '2TNDI', 'acetylpyridine', 'nanosheets', 'nanosphere', 'helices', 'PF6', 'N', 'amino', 'nanocones', 'nw;', 'FIMG', 'FITG', 'tran3', 'T2)', 'OMe', 'DFBT:PCBM', 'PDI2', 'Ep', 'vTA', '(F)', 'TrFE', 'b', 'Ph', 'methoxythiophenol', 'T2);', 'PYBrZnPor', '5a', '5b', '5c', '5d', '5e', '5g', '0S', '2S', '3S', '4S', 'PFE1', 'PFE2', 'PFE3', 'PFE4', 'S', 'Bphen', 'derivative10', 'derivative11', 'derivative12', 'ethyl', 'TAZ', '2T', 'TFSI', 'macroporous', '8', 'TT', 'Na', 'Aminopropyl)trimethoxysilane', 'Se', 'nanoplatelets', '63', '62', 'PEIE', 'NH2', 'PCBOE', 'mp;', 'PCBSD', 'Pyr', 'nanowells', 'nan

In [ ]:
abb_full = {'c':'carbon',
            'mp':'', 
            'np':'nanoparticle',
            'nw':'nanowire',
            'nt':'nanotexture',
            'ns':'',
            'nc':''}